## Equipping Our Dataset for Success: The 80/20 Pidgin-English Mix Strategy

### The Goal: Why We Are Modifying Our Dataset

Our primary goal is to perform Continued Pretraining (CPT) to adapt a large, English-centric language model (like Llama or Gemma) so it can understand and generate Nigerian Pidgin fluently and intelligently.

However, recent research has shown that simply training a model on a new, monolingual dataset can be counterproductive. Based on the critical findings in the paper **"Emergent Abilities of Large Language Models under Continued Pretraining for Language Adaptation"** ([arXiv:2506.00288](https://arxiv.org/abs/2506.00288)), we are strategically modifying our dataset to avoid a phenomenon known as **catastrophic forgetting**.

> **In short:** We are mixing a small amount of English data into our Pidgin dataset to ensure our final model doesn't just learn to *speak* Pidgin, but also retains its ability to *think* and *reason*.

### Key Insights from the Research

The paper reveals a crucial problem with monolingual CPT:

1.  **The Perplexity Trap:** When a model is trained exclusively on a new language (e.g., Basque in the paper, or Nigerian Pidgin in our case), it quickly achieves a low perplexity score. This means it becomes very good at predicting the next word in that language. However, this metric is deceptive, as the model's performance on actual downstream tasks (like question-answering) plummets.

2.  **Forgetting How to Learn:** The core issue is that the drastic "distribution shift" from English to a new language causes the model to forget its fundamental **In-Context Learning (ICL)** capabilities. It loses the emergent ability to understand patterns from a few examples, which is a cornerstone of modern LLMs.

3.  **The Solution: An "Anchor" Language:** The paper's key finding is that including a small portion of English data (around **20%**) in the training mixture acts as a regularizer. This English data "anchors" the model's parameters, preventing the drastic shifts that lead to catastrophic forgetting.

### Our Implementation: Creating the 80/20 Mix

To make our `orpheus-pidgin-cpt` dataset "well-equipped" for this task, we are directly implementing the paper's recommended 80/20 mixing strategy. While our initial attempt was to use the `EleutherAI/pile` dataset, we encountered an SSL certificate error with its hosting server. As a robust and high-quality alternative, we are now using the C4 dataset.

*   **80% Nigerian Pidgin:** We use our entire custom dataset from the `pidgin_cpt.jsonl` file. This is the primary data for teaching the model the vocabulary, grammar, and nuances of Nigerian Pidgin.

*   **20% English:** We are programmatically sampling a corresponding amount of high-quality English text from the **`allenai/c4`** dataset (Colossal Cleaned Common Crawl). C4 is another massive, diverse corpus that serves as an excellent source of general English text for our regularization needs.

The code in the following cells will combine these two sources, shuffle them thoroughly, and create a new training file: `pidgin_english_cpt_80_20.jsonl`. This final, mixed dataset is what we will upload to the Hugging Face Hub, as it is now properly structured to produce a capable and intelligent Nigerian Pidgin language model.

In [1]:
!pip install huggingface_hub -q
!pip install wandb datasets -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.1 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incomp

In [2]:
# ==============================================================================
# CELL 2: Login to Hugging Face and Weights & Biases
# ==============================================================================
import wandb
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

# --- PRE-REQUISITES ---
# 1. In your Kaggle notebook, go to "Add-ons" > "Secrets".
# 2. Add your Hugging Face WRITE token with the label "HUGGINGFACE_API_KEY".
# 3. Add your W&B API key with the label "wandb_api_key".
# 4. This keeps your keys secure and private.
# ----------------------

# --- Hugging Face Login ---
print("--- Attempting Hugging Face Login ---")
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HUGGINGFACE_API_KEY")
    login(token=hf_token)
    print("✅ Successfully logged into Hugging Face.")
except Exception as e:
    print("Could not log into Hugging Face. Please ensure the 'HUGGINGFACE_API_KEY' secret is set.")
    print(f"Error: {e}")

# --- Weights & Biases Login ---
print("\n--- Attempting Weights & Biases Login ---")
try:
    user_secrets = UserSecretsClient()
    wandb_api_key = user_secrets.get_secret("wandb_api_key")
    wandb.login(key=wandb_api_key)
    print("✅ Successfully logged into Weights & Biases.")
except Exception as e:
    print("Could not log into W&B. Please ensure the 'wandb_api_key' secret is set.")
    print(f"Error: {e}")

--- Attempting Hugging Face Login ---
✅ Successfully logged into Hugging Face.

--- Attempting Weights & Biases Login ---


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jdmasciano2 (jdmasciano2-university-of-lagos) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ Successfully logged into Weights & Biases.


In [3]:
jsonl_data = """
{"text": "How you dey? I dey fine my brother."}
{"text": "Di woman dey fry akara for morning market."}
{"text": "Pikin no gree sleep last night, e dey cry well well."}
{"text": "No matter how long journey be, e go end one day."}
{"text": "Abeg close di window before rain enter house."}
{"text": "Driver no wan wait, e just press horn waka."}
{"text": "Na football dey make di youth happy for dis area."}
{"text": "Dem no pay di worker since last month."}
{"text": "Rice and beans dey sweet if you add small stew."}
{"text": "Water no get enemy, na true talk."}
{"text": "Di boy carry basket of orange go sell for junction."}
{"text": "Dem dey talk say tomorrow meeting go long."}
{"text": "She dey dance for party like say na she be star."}
{"text": "Aboki dey roast corn for street corner."}
{"text": "Di old man dey tell story under mango tree."}
{"text": "Who no work no go chop, na wetin elders dey talk."}
{"text": "Rain begin fall when we dey waka go church."}
{"text": "Pikin dey learn how to read and write for school."}
{"text": "Plenty people gather for field to watch match."}
{"text": "No use am shine teeth, action better pass talk."}
{"text": "Mama send me go buy pepper for junction."}
{"text": "Small small, bird dey build nest."}
{"text": "Di shop no get change, so dem give me sweet instead."}
{"text": "No let small wahala spoil your day."}
{"text": "Motor break down for road, everybody come down push am."}
{"text": "Di musician voice sweet well well."}
{"text": "She dey carry heavy load but she no complain."}
{"text": "Who chop last night soup go still chop am today."}
{"text": "Go slow don hold di road again."}
{"text": "Man dey hustle from morning till night."}
{"text": "Di teacher dey explain mathematics for class."}
{"text": "Everybody dey laugh for comedy show."}
{"text": "Di goat run comot from backyard."}
{"text": "Abeg help me hold bag make I tie my shoe."}
{"text": "Hospital dey far from village, people dey suffer."}
{"text": "Small pikin dey play with sand for ground."}
{"text": "Na better day to start new business."}
{"text": "E don tey wey we no see rain like dis."}
{"text": "Di tailor dey sew fine cloth for customer."}
{"text": "Make we no fight, peace good pass war."}
{"text": "Chicken dey scratch ground dey find food."}
{"text": "Di man buy generator because light no dey."}
{"text": "Woman dey pray for better life for her children."}
{"text": "Abeg shift, make I sit small for chair."}
{"text": "Who get patience go chop ripe fruit."}
{"text": "Pikin dey cry say im want biscuit."}
{"text": "Di market full of people dis Saturday."}
{"text": "Better soup na money kill am."}
{"text": "Friendship strong pass money sometimes."}
{"text": "We no fit live without water."}
{"text": "No dey underrate anybody wey quiet."}
{"text": "Pikin wey sabi respect go grow well."}
{"text": "Di boy dey wash him papa motor for compound."}
{"text": "No be every smile mean say person dey happy."}
{"text": "Di old woman dey sell banana for roadside."}
{"text": "Na small small we dey learn big thing."}
{"text": "She carry basket of tomato go give her aunty."}
{"text": "Man wey no fear work go chop better food."}
{"text": "No dey judge book by cover."}
{"text": "I go wait for you under big tree."}
{"text": "Time dey go, make we no waste am."}
{"text": "Goat dey waka enter neighbour farm."}
{"text": "Dem don start new bridge project for town."}
{"text": "No dey put hand for wetin you no sabi."}
{"text": "Motor dey blow horn anyhow for street."}
{"text": "Sun hot today, everywhere dey dry."}
{"text": "Abeg pass me salt make I cook soup."}
{"text": "Life na turn by turn, today fit be your own."}
{"text": "Rain dey beat me when I dey waka home."}
{"text": "Naija music dey sweet for party."}
{"text": "Teacher dey mark exam script for office."}
{"text": "Di farmer dey clear land for planting."}
{"text": "No gree carry another person wahala for head."}
{"text": "Children dey play football for compound."}
{"text": "Man no fit serve two master at once."}
{"text": "She dey write letter to her mama."}
{"text": "Pikin chop too much sugar, e come sick."}
{"text": "Di hunter bring bushmeat come house."}
{"text": "No matter how river long, e get end."}
{"text": "Pikin dey carry bucket of water for head."}
{"text": "No dey delay if chance show."}
{"text": "Bread and tea na better breakfast."}
{"text": "Who dey sleep for class no go sabi pass exam."}
{"text": "Di bicycle tyre burst for road."}
{"text": "Mama dey teach her daughter how to cook soup."}
{"text": "Abeg put radio small small, noise too much."}
{"text": "Market woman dey shout price for customer."}
{"text": "No dey fight over small thing."}
{"text": "Work hard today make tomorrow soft."}
{"text": "No dey use anger spoil friendship."}
{"text": "Dog dey bark every night for street."}
{"text": "Pikin smile fit change person mood."}
{"text": "She dey wait for bus wey go carry her go town."}
{"text": "Na money dey cause plenty wahala."}
{"text": "Rain fall scatter ground for market."}
{"text": "Di man dey teach him son how to fish."}
{"text": "We dey wait for NEPA to bring light."}
{"text": "Monkey no fine but e mama like am."}
{"text": "Di pastor dey preach about love for church."}
{"text": "Small chance fit change person life."}
{"text": "E good make we dey thank God for small things."}
{"text": "Morning breeze dey fresh for village side."}
{"text": "Di boy no gree share him biscuit with anybody."}
{"text": "Who laugh last laugh best."}
{"text": "Di fisherman waka early morning go river."}
{"text": "No dey waste food, e fit help another person."}
{"text": "Market dey full every Saturday morning."}
{"text": "Di woman dey plait hair for her customer."}
{"text": "Na patience dey make plantain ripe."}
{"text": "She no wan talk, she just dey nod head."}
{"text": "Abeg buy garri before e finish for shop."}
{"text": "Pikin wey no hear word go learn with experience."}
{"text": "Sun dey shine, everybody dey happy."}
{"text": "Dog dey follow him master up and down."}
{"text": "Di boy borrow pen from him friend for exam."}
{"text": "She dey practice dance step for wedding."}
{"text": "Better friend go stand with you for hard time."}
{"text": "Di road no good, motor dey enter pothole."}
{"text": "Small pikin dey laugh when e see balloon."}
{"text": "Rain dey make everywhere green for farm."}
{"text": "Di tailor no gree deliver cloth on time."}
{"text": "Everyday no be Christmas."}
{"text": "Di soldier dey march for parade ground."}
{"text": "Di woman dey sing lullaby for her baby."}
{"text": "No dey forget person wey help you."}
{"text": "Di shopkeeper dey count change slowly."}
{"text": "Monkey dey jump for tree branch."}
{"text": "No dey complain if you no put effort."}
{"text": "Di hunter dey prepare trap for bush."}
{"text": "Children dey clap hand for assembly."}
{"text": "She dey fry yam for evening snack."}
{"text": "Bicycle rider dey ring bell for road."}
{"text": "No dey compare yourself with another person."}
{"text": "Di old man dey sleep for veranda chair."}
{"text": "Na true friend go stay when money finish."}
{"text": "Pikin dey draw picture with chalk for ground."}
{"text": "Di carpenter dey hammer nail for wood."}
{"text": "Life no hard if person get good plan."}
{"text": "Everybody dey clap when singer finish song."}
{"text": "No matter how e be, hope still dey."}
{"text": "Di bus conductor dey shout bus stop."}
{"text": "She carry small baby for back."}
{"text": "Person wey dey humble go reach far."}
{"text": "Abeg make we sit down talk small."}
{"text": "Pikin wey chop well go grow strong."}
{"text": "Di chief dey give speech for village square."}
{"text": "No let anger control your life."}
{"text": "Rain dey fall heavy for midnight."}
{"text": "Di student dey write test for classroom."}
{"text": "Fish dey swim for river freely."}
{"text": "Who get ear make e hear."}
{"text": "Na who no work, no go chop."}
{"text": "Di sun hot well-well for afternoon."}
{"text": "My mama don go market go buy pepper."}
{"text": "Tomorrow we go paint di new shop."}
{"text": "Dog dey bark since morning."}
{"text": "Dem no gree make we enter without ID card."}
{"text": "Teacher tell us make we read chapter three."}
{"text": "E dey plan to start small business next week."}
{"text": "Di sky clear, breeze dey blow soft-soft."}
{"text": "She dey learn how to drive for driving school."}
{"text": "Na today we go finish di roof work."}
{"text": "Baby dey sleep peacefully for cradle."}
{"text": "My shoe tear small but e still manage."}
{"text": "Na only patience fit make dream come true."}
{"text": "Motor break down for middle of express road."}
{"text": "We go carry yam go village this weekend."}
{"text": "No be every smile mean better thing."}
{"text": "Rain don start, make you carry umbrella."}
{"text": "Person wey dey humble go go far."}
{"text": "E chop too much, now sleep dey catch am."}
{"text": "Dem dey rehearse drama for school hall."}
{"text": "Goat enter garden go chop cassava leaf."}
{"text": "I hear say new shop don open for junction."}
{"text": "E dey brush teeth before e chop every morning."}
{"text": "We gather for church go do thanksgiving."}
{"text": "Monkey wey no get tail no fit hide for tree."}
{"text": "Dem go share food after meeting."}
{"text": "No use anger spoil wetin you don build."}
{"text": "Di soup sweet pass yesterday own."}
{"text": "My papa like watch football every weekend."}
{"text": "No carry another person wahala for head."}
{"text": "I see rainbow for sky after rain stop."}
{"text": "Dem don begin repair di old bridge."}
{"text": "I wan learn how to bake cake."}
{"text": "Better plan dey save person from wahala."}
{"text": "Na hard work dey bring success."}
{"text": "We dey wait bus since morning."}
{"text": "No put salt too much for di rice."}
{"text": "E dey greet everybody wey pass for street."}
{"text": "Children dey play ball for field."}
{"text": "I don wash clothes finish."}
{"text": "Dem dey clean gutter for front of house."}
{"text": "Light don come back after three days."}
{"text": "Di doctor check my temperature this morning."}
{"text": "We go visit grandma for village soon."}
{"text": "No let pride block your blessing."}
{"text": "E buy new phone yesterday for market."}
{"text": "Na joy full my heart today."}
{"text": "Di wind dey blow from east side."}
{"text": "She dey write letter give her friend."}
{"text": "Pikin dey cry because e hungry."}
{"text": "Dem dey plant maize for farm now."}
{"text": "I go iron cloth before I go work."}
{"text": "Di goat jump fence enter neighbor compound."}
{"text": "We go cook rice and beans today."}
{"text": "No use loud voice talk for library."}
{"text": "Rain dey fall small-small since morning."}
{"text": "Fish dey fry for kitchen."}
{"text": "Di man dey fix tire for roadside."}
{"text": "Na good thing to dey help people."}
{"text": "We dey wait result from exam board."}
{"text": "Cow dey graze for open field."}
{"text": "Dem dey build new classroom for school."}
{"text": "No forget to lock door before you comot."}
{"text": "Na truth go always stand last."}
{"text": "E no easy but we go try again."}
{"text": "Dem carry basket full of mango."}
{"text": "We go fetch water from borehole later."}
{"text": "Na my sister cook this sweet stew."}
{"text": "Di boy dey play drum for church."}
{"text": "She dey read novel under mango tree."}
{"text": "We go plant tree make compound fine."}
{"text": "No follow bad gang for street."}
{"text": "Na patience dey open better door."}
{"text": "Di wind strong last night."}
{"text": "Di tailor don sew my cloth finish."}
{"text": "Na faith make me still dey hope."}
{"text": "We go wash plate together after food."}
{"text": "No forget your bag for bus stop."}
{"text": "Di teacher dey mark script for office."}
{"text": "Pikin dey dance for children party."}
{"text": "Rain scatter clothes wey I dry for outside."}
{"text": "We dey go farm early tomorrow morning."}
{"text": "Na small-small person dey learn big thing."}
{"text": "E dey fix bulb wey blow yesterday."}
{"text": "No let lie destroy your name."}
{"text": "Dem go start festival next month."}
{"text": "My uncle dey teach computer for center."}
{"text": "We chop pounded yam for dinner yesterday."}
{"text": "No fear failure, na lesson e be."}
{"text": "We go paint di wall white next week."}
{"text": "She dey sing for choir every Sunday."}
{"text": "No let small success enter your head."}
{"text": "E dey brush teeth for bathroom now."}
{"text": "Di moon dey shine bright this night."}
{"text": "Dem dey dance cultural dance for hall."}
{"text": "I dey go buy bread for mama Bisi shop."}
{"text": "Na good habit to dey greet people early."}
{"text": "She cook jollof rice wey sweet well."}
{"text": "We go wash motor after rain stop."}
{"text": "Dem dey plan community clean-up."}
{"text": "I dey pack load go school hostel."}
{"text": "Na joy to see old friend again."}
{"text": "E carry chair go fix for backyard."}
{"text": "Na who patient dey collect reward last."}
{"text": "We dey repair roof before rainy season come."}
{"text": "No forget where you come from."}
{"text": "Di chicken escape from cage today."}
{"text": "She dey plait hair for salon now."}
{"text": "E dey check time every five minutes."}
{"text": "We go play football after school close."}
{"text": "Di new movie sweet pass expectation."}
{"text": "E don finish assignment before night reach."}
{"text": "Dem dey fry akara for junction."}
{"text": "Na wisdom dey make man calm for wahala."}
{"text": "She dey laugh every time she happy."}
{"text": "We go chop together for evening."}
{"text": "No use anger talk wetin you go regret."}
{"text": "Di cloth wey e sew fine well."}
{"text": "Dem don fix street light for our area."}
{"text": "We go attend wedding next Saturday."}
{"text": "Di river near our house don full."}
{"text": "She dey sweep compound every morning."}
{"text": "No forget to thank person wey help you."}
{"text": "We dey share story for night before sleep."}
{"text": "Di breeze wey blow today sweet well."}
{"text": "Na unity dey make community strong."}
{"text": "Dem dey prepare soup for family meeting."}
{"text": "We dey fix chair for event hall."}
{"text": "I go write letter to my cousin soon."}
{"text": "Na hope dey keep heart strong."}
{"text": "She dey carry basket go market every Friday."}
{"text": "Na who no work, no go chop."}
{"text": "Di sun hot well-well for afternoon."}
{"text": "My mama don go market go buy pepper."}
{"text": "Tomorrow we go paint di new shop."}
{"text": "Dog dey bark since morning."}
{"text": "Dem no gree make we enter without ID card."}
{"text": "Teacher tell us make we read chapter three."}
{"text": "E dey plan to start small business next week."}
{"text": "Di sky clear, breeze dey blow soft-soft."}
{"text": "She dey learn how to drive for driving school."}
{"text": "Na today we go finish di roof work."}
{"text": "Baby dey sleep peacefully for cradle."}
{"text": "My shoe tear small but e still manage."}
{"text": "Na only patience fit make dream come true."}
{"text": "Motor break down for middle of express road."}
{"text": "We go carry yam go village this weekend."}
{"text": "No be every smile mean better thing."}
{"text": "Rain don start, make you carry umbrella."}
{"text": "Person wey dey humble go go far."}
{"text": "E chop too much, now sleep dey catch am."}
{"text": "Dem dey rehearse drama for school hall."}
{"text": "Goat enter garden go chop cassava leaf."}
{"text": "I hear say new shop don open for junction."}
{"text": "E dey brush teeth before e chop every morning."}
{"text": "We gather for church go do thanksgiving."}
{"text": "Monkey wey no get tail no fit hide for tree."}
{"text": "Dem go share food after meeting."}
{"text": "No use anger spoil wetin you don build."}
{"text": "Di soup sweet pass yesterday own."}
{"text": "My papa like watch football every weekend."}
{"text": "No carry another person wahala for head."}
{"text": "I see rainbow for sky after rain stop."}
{"text": "Dem don begin repair di old bridge."}
{"text": "I wan learn how to bake cake."}
{"text": "Better plan dey save person from wahala."}
{"text": "Na hard work dey bring success."}
{"text": "We dey wait bus since morning."}
{"text": "No put salt too much for di rice."}
{"text": "E dey greet everybody wey pass for street."}
{"text": "Children dey play ball for field."}
{"text": "I don wash clothes finish."}
{"text": "Dem dey clean gutter for front of house."}
{"text": "Light don come back after three days."}
{"text": "Di doctor check my temperature this morning."}
{"text": "We go visit grandma for village soon."}
{"text": "No let pride block your blessing."}
{"text": "E buy new phone yesterday for market."}
{"text": "Na joy full my heart today."}
{"text": "Di wind dey blow from east side."}
{"text": "She dey write letter give her friend."}
{"text": "Pikin dey cry because e hungry."}
{"text": "Dem dey plant maize for farm now."}
{"text": "I go iron cloth before I go work."}
{"text": "Di goat jump fence enter neighbor compound."}
{"text": "We go cook rice and beans today."}
{"text": "No use loud voice talk for library."}
{"text": "Rain dey fall small-small since morning."}
{"text": "Fish dey fry for kitchen."}
{"text": "Di man dey fix tire for roadside."}
{"text": "Na good thing to dey help people."}
{"text": "We dey wait result from exam board."}
{"text": "Cow dey graze for open field."}
{"text": "Dem dey build new classroom for school."}
{"text": "No forget to lock door before you comot."}
{"text": "Na truth go always stand last."}
{"text": "E no easy but we go try again."}
{"text": "Dem carry basket full of mango."}
{"text": "We go fetch water from borehole later."}
{"text": "Na my sister cook this sweet stew."}
{"text": "Di boy dey play drum for church."}
{"text": "She dey read novel under mango tree."}
{"text": "We go plant tree make compound fine."}
{"text": "No follow bad gang for street."}
{"text": "Na patience dey open better door."}
{"text": "Di wind strong last night."}
{"text": "Di tailor don sew my cloth finish."}
{"text": "Na faith make me still dey hope."}
{"text": "We go wash plate together after food."}
{"text": "No forget your bag for bus stop."}
{"text": "Di teacher dey mark script for office."}
{"text": "Pikin dey dance for children party."}
{"text": "Rain scatter clothes wey I dry for outside."}
{"text": "We dey go farm early tomorrow morning."}
{"text": "Na small-small person dey learn big thing."}
{"text": "E dey fix bulb wey blow yesterday."}
{"text": "No let lie destroy your name."}
{"text": "Dem go start festival next month."}
{"text": "My uncle dey teach computer for center."}
{"text": "We chop pounded yam for dinner yesterday."}
{"text": "No fear failure, na lesson e be."}
{"text": "We go paint di wall white next week."}
{"text": "She dey sing for choir every Sunday."}
{"text": "No let small success enter your head."}
{"text": "E dey brush teeth for bathroom now."}
{"text": "Di moon dey shine bright this night."}
{"text": "Dem dey dance cultural dance for hall."}
{"text": "I dey go buy bread for mama Bisi shop."}
{"text": "Na good habit to dey greet people early."}
{"text": "She cook jollof rice wey sweet well."}
{"text": "We go wash motor after rain stop."}
{"text": "Dem dey plan community clean-up."}
{"text": "I dey pack load go school hostel."}
{"text": "Na joy to see old friend again."}
{"text": "E carry chair go fix for backyard."}
{"text": "Na who patient dey collect reward last."}
{"text": "We dey repair roof before rainy season come."}
{"text": "No forget where you come from."}
{"text": "Di chicken escape from cage today."}
{"text": "She dey plait hair for salon now."}
{"text": "E dey check time every five minutes."}
{"text": "We go play football after school close."}
{"text": "Di new movie sweet pass expectation."}
{"text": "E don finish assignment before night reach."}
{"text": "Dem dey fry akara for junction."}
{"text": "Na wisdom dey make man calm for wahala."}
{"text": "She dey laugh every time she happy."}
{"text": "We go chop together for evening."}
{"text": "No use anger talk wetin you go regret."}
{"text": "Di cloth wey e sew fine well."}
{"text": "Dem don fix street light for our area."}
{"text": "We go attend wedding next Saturday."}
{"text": "Di river near our house don full."}
{"text": "She dey sweep compound every morning."}
{"text": "No forget to thank person wey help you."}
{"text": "We dey share story for night before sleep."}
{"text": "Di breeze wey blow today sweet well."}
{"text": "Na unity dey make community strong."}
{"text": "Dem dey prepare soup for family meeting."}
{"text": "We dey fix chair for event hall."}
{"text": "I go write letter to my cousin soon."}
{"text": "Na hope dey keep heart strong."}
{"text": "She dey carry basket go market every Friday."}
{"text": "We dey wait bus since early morning."}
{"text": "Rain don scatter all di sand wey dem pour for road."}
{"text": "Mama dey sing as she dey cook soup for kitchen."}
{"text": "Na only true friend go stay when wahala come."}
{"text": "Dem dey sell fresh fish for river side."}
{"text": "I go wash my cloth later when sun come out."}
{"text": "Dog dey run follow car wey pass street."}
{"text": "We go paint di house blue after renovation."}
{"text": "E dey help him papa for shop every evening."}
{"text": "No put too much pepper for soup oh!"}
{"text": "Di teacher talk say exam go start next week."}
{"text": "I go send message to my sister tomorrow."}
{"text": "We dey arrange chair for meeting hall."}
{"text": "No dey shout for person face anyhow."}
{"text": "Pikin dey happy as e see new toy."}
{"text": "Dem dey clean street for environmental day."}
{"text": "My neighbor buy new generator last week."}
{"text": "Di rain too much, everywhere don flood."}
{"text": "No forget your water bottle before you comot."}
{"text": "We go celebrate birthday for Sunday."}
{"text": "E dey fry plantain for corner of kitchen."}
{"text": "No rush, take your time do am well."}
{"text": "Dem don repair di borehole for community."}
{"text": "She dey comb her hair before she go out."}
{"text": "I don finish reading my novel for night."}
{"text": "E buy shoe wey fine well for boutique."}
{"text": "No worry, everything go set soon."}
{"text": "Di small pikin dey learn how to write ABC."}
{"text": "We go meet for junction by five o’clock."}
{"text": "She dey arrange her clothes inside box."}
{"text": "Na laugh full my mouth as I see di play."}
{"text": "Dem don build new classroom for primary school."}
{"text": "E dey save money small-small every month."}
{"text": "No use lie cover another lie."}
{"text": "We go cook egusi soup later this evening."}
{"text": "Di wind blow my cloth enter gutter."}
{"text": "Pikin dey run around compound dey play."}
{"text": "Na who get plan go reach destination."}
{"text": "Dem dey practice song for choir practice."}
{"text": "I go fetch water for borehole later today."}
{"text": "E buy new phone for Black Friday promo."}
{"text": "We dey fix chair wey break during party."}
{"text": "Na morning breeze dey make place cool."}
{"text": "Di pastor preach about peace and unity."}
{"text": "No use person downfall take make jest."}
{"text": "We go travel go Lagos next month."}
{"text": "She dey arrange flower for decoration."}
{"text": "Di market full well-well today."}
{"text": "I go check doctor tomorrow for hospital."}
{"text": "Dem dey set tent for wedding reception."}
{"text": "No forget umbrella before you comot house."}
{"text": "E dey whistle as e dey waka for street."}
{"text": "We chop yam and egg for breakfast."}
{"text": "Na small world, I jam my friend for bus."}
{"text": "Dem dey play loud music for corner bar."}
{"text": "She dey learn make-up from online video."}
{"text": "I go sweep compound before rain start."}
{"text": "Di car tire burst for middle of road."}
{"text": "We dey go church early tomorrow morning."}
{"text": "Dem dey cook jollof rice for party now."}
{"text": "E wear white cloth go naming ceremony."}
{"text": "No dey give promise wey you no fit keep."}
{"text": "We go carry fruit go give our teacher."}
{"text": "Na joy to see pikin wey dey smile."}
{"text": "Dem dey build new bridge for river crossing."}
{"text": "I don charge my phone full for night."}
{"text": "Na rain make everywhere cold this evening."}
{"text": "Di student dey wait for exam result."}
{"text": "We go fix table for dining room later."}
{"text": "She buy pepper, tomato, and onion for market."}
{"text": "Dem dey wash car for compound now."}
{"text": "No rush success, take am step by step."}
{"text": "E dey help carry chair for old woman."}
{"text": "We go go cinema watch film on Friday."}
{"text": "Na water dey calm body when heat too much."}
{"text": "Dem dey rehearse dance for school program."}
{"text": "I go buy biscuit for my younger brother."}
{"text": "Na truth dey make person free."}
{"text": "E dey cut grass for backyard."}
{"text": "We dey count days till Christmas reach."}
{"text": "No dey envy person blessing."}
{"text": "She dey wash plate for kitchen now."}
{"text": "E don finish assignment before dinner."}
{"text": "We go visit uncle for Benin next week."}
{"text": "Na better thing to dey keep promise."}
{"text": "Di rain wash di dust commot from road."}
{"text": "Dem dey play draft under mango tree."}
{"text": "I go clean window before mama come back."}
{"text": "Na music dey make my mood better."}
{"text": "E dey read newspaper every morning."}
{"text": "We go collect certificate on graduation day."}
{"text": "No forget to say thank you when person help you."}
{"text": "Dem dey discuss politics for radio show."}
{"text": "Di woman dey roast corn for roadside."}
{"text": "We dey plan surprise party for papa birthday."}
{"text": "Na hard work dey make success sweet."}
{"text": "E carry umbrella even though sun dey shine."}
{"text": "We go cook okro soup with stockfish."}
{"text": "No dey argue with person wey no wan reason."}
{"text": "Dem don finish painting di compound wall."}
{"text": "She dey type report for office computer."}
{"text": "I go buy airtime before network go off."}
{"text": "Di shop open early today."}
{"text": "We dey hear thunder for far distance."}
{"text": "E dey fix radio wey spoil yesterday."}
{"text": "Na good neighbor dey make life easy."}
{"text": "She dey sew new gown for Christmas."}
{"text": "We go chop suya later for junction."}
{"text": "No let laziness hold your destiny down."}
{"text": "Dem dey test microphone before event start."}
{"text": "E dey carry file go submit for office."}
{"text": "We dey tidy up before guest go arrive."}
{"text": "Di bird dey sing sweet song for tree top."}
{"text": "She dey clean glass cup one by one."}
{"text": "We go attend choir rehearsal in evening."}
{"text": "Na patience and prayer dey change story."}
{"text": "Dem dey clap as dancer enter stage."}
{"text": "E dey smile as e remember old gist."}
{"text": "We go go beach next Sunday to relax."}
{"text": "Na good spirit dey bring peace for heart."}
{"text": "She dey feed baby with small spoon."}
{"text": "E dey knock door before e enter room."}
{"text": "We dey do morning devotion every day."}
{"text": "Na joy full everybody for wedding ground."}
{"text": "Di driver dey rest after long journey."}
{"text": "E buy cold pure water for junction."}
{"text": "We go share food for less privileged children."}
{"text": "No dey talk too much for serious matter."}
{"text": "Dem dey arrange chair for youth meeting."}
{"text": "She dey iron cloth for work tomorrow."}
{"text": "I go write my plan for new notebook."}
{"text": "We dey celebrate independence for stadium."}
{"text": "Na only wise person fit keep secret."}
{"text": "E dey pack yam for storage room."}
{"text": "We go pray together before we sleep."}
{"text": "Na laughter dey heal broken heart."}
{"text": "Dem dey open new bakery for town."}
{"text": "She dey bake meat pie for customer order."}
{"text": "E dey call him mama every Sunday evening."}
{"text": "We go fetch firewood before night fall."}
{"text": "Di boy dey sing along with radio."}
{"text": "No forget your key for inside house again."}
{"text": "We dey check time make we no late for work."}
{"text": "E dey write exam with confidence."}
{"text": "Na faith and effort dey move mountain."}
{"text": "She dey laugh as her friend dey gist."}
{"text": "We go sweep classroom before lesson start."}
{"text": "E dey fix roof wey leak during rain."}
{"text": "No dey gossip about person wey no dey around."}
{"text": "Dem dey set table for family lunch."}
{"text": "I go rest small after long waka."}
{"text": "Na better thing to dey share knowledge."}
{"text": "She dey peel orange for her pikin."}
{"text": "We go tidy room before visitor arrive."}
{"text": "Na good day to start something new."}
{"text": "Dem dey sell roasted yam for roadside stand."}
{"text": "E dey wash hand before e chop food."}
{"text": "We dey thank God for another day."}
{"text": "Na respect make relationship sweet."}
{"text": "She dey fry puff-puff for morning market."}
{"text": "We go plant maize when rain start again."}
{"text": "E dey pack book inside bag for school."}
{"text": "No forget to check your change for shop."}
{"text": "No carry another person wahala for head."}
{"text": "She dey sell puff puff for junction."}
{"text": "Better soup na money kill am."}
{"text": "The teacher dey mark test with red pen."}
{"text": "No matter how rain fall, sun go still shine."}
{"text": "Pikin dey play football for field."}
{"text": "I go come back once I finish my work."}
{"text": "The goat don escape from backyard again."}
{"text": "She dey comb her pikin hair every morning."}
{"text": "The driver dey horn make people clear road."}
{"text": "No dey laugh person when him fall."}
{"text": "Di man dey drink cold water for shade."}
{"text": "When hunger catch person, even garri go sweet."}
{"text": "She tie wrapper well before she enter market."}
{"text": "You go chop breakfast before you go work?"}
{"text": "Rain don wash away all the sand for compound."}
{"text": "The woman dey sweep church compound early morning."}
{"text": "Pikin wey stubborn go collect cane."}
{"text": "Abeg shift small make I pass."}
{"text": "Who dey knock for gate this time?"}
{"text": "The mechanic dey under motor dey check engine."}
{"text": "I no like noise, make una calm down small."}
{"text": "Na patience dey bring better result."}
{"text": "The man dey fix generator for night."}
{"text": "Abeg help me hold bag make I tie lace."}
{"text": "Who carry my spoon from kitchen?"}
{"text": "The cat dey sleep for chair like king."}
{"text": "No be everything eye see mouth go talk."}
{"text": "Di old woman dey peel cassava for bowl."}
{"text": "When day break, cock go crow."}
{"text": "You no fit hide smoke when fire dey."}
{"text": "The boy don climb mango tree again."}
{"text": "I go reach there before evening time."}
{"text": "The small girl dey pluck flower for garden."}
{"text": "Na food dey make body strong."}
{"text": "Rain dey fall but sun still dey shine small."}
{"text": "Di man dey sharpen cutlass for farm work."}
{"text": "Pikin dey sleep, make una no make noise."}
{"text": "The tailor dey measure cloth for customer."}
{"text": "Who dey play music for back of house?"}
{"text": "The woman dey fan firewood to cook rice."}
{"text": "No dey rush life, everything get time."}
{"text": "The old man dey watch news for TV."}
{"text": "Breeze blow my cap commot for head."}
{"text": "The hunter don return with bushmeat."}
{"text": "Abeg open window make air enter room."}
{"text": "The chicken dey run from knife."}
{"text": "Na true word go stand forever."}
{"text": "The boy dey whistle as he waka go school."}
{"text": "Di shopkeeper don close store for today."}
{"text": "The woman dey fry akara for roadside."}
{"text": "No dey talk too much for where elders dey."}
{"text": "The boy dey sweep compound before school."}
{"text": "Na small drop of water dey full bucket."}
{"text": "She dey wash plate while music dey play."}
{"text": "The rain stop small but sky still dark."}
{"text": "Who dey knock for back door?"}
{"text": "The tailor don finish my native cloth."}
{"text": "No be by shouting person dey win argument."}
{"text": "The driver park motor for filling station."}
{"text": "Goat don jump fence enter neighbor farm."}
{"text": "The old man dey tell story under mango tree."}
{"text": "Make we gather chop before food cold."}
{"text": "She buy tomato and pepper from market."}
{"text": "No use anger spoil better thing."}
{"text": "The bus conductor dey call passengers loud."}
{"text": "Di boy dey repair shoe for roadside."}
{"text": "She dey dance for birthday party."}
{"text": "Monkey see monkey do, na so trouble start."}
{"text": "The girl dey sing sweet song for church."}
{"text": "Abeg close gate before goat enter house."}
{"text": "The farmer dey rest under palm tree."}
{"text": "Di old woman dey roast corn for evening."}
{"text": "Who carry my slippers go outside?"}
{"text": "No matter how night dark, morning go come."}
{"text": "The pikin dey wash hand before chop."}
{"text": "Na small thing dey cause big wahala sometimes."}
{"text": "She dey braid hair for her sister."}
{"text": "The teacher dey share exercise book."}
{"text": "No dey use person weakness laugh am."}
{"text": "The boy dey drink zobo with biscuit."}
{"text": "Na love dey hold family together."}
{"text": "The man don fix bulb for corridor."}
{"text": "Pikin wey stubborn no go hear warning."}
{"text": "The tailor dey iron cloth for customer."}
{"text": "No dey forget say tomorrow still dey."}
{"text": "The woman dey peel yam for pot."}
{"text": "Rain dey fall since morning no stop."}
{"text": "Abeg borrow me pen make I sign form."}
{"text": "The old papa dey read newspaper slowly."}
{"text": "Dog dey bark for midnight anyhow."}
{"text": "No be who first start race dey win always."}
{"text": "The girl dey toast bread for breakfast."}
{"text": "Sun dey hot, make we find shade rest small."}
{"text": "Who hide remote control for parlor?"}
{"text": "The small boy dey count his toy cars."}
{"text": "Na respect dey open better door for person."}
{"text": "The man dey polish shoe before work."}
{"text": "No forget to lock gate before you sleep."}
{"text": "The pikin dey draw flower for wall with chalk."}
{"text": "Mama dey cook egusi soup with plenty fish."}
{"text": "The boy carry bucket go fetch water for stream."}
{"text": "No let small success make you proud too quick."}
{"text": "The teacher dey mark exam papers for night."}
{"text": "Na good friend go tell you truth even if e pain."}
{"text": "The carpenter dey fix broken chair for veranda."}
{"text": "Who chop all the meat for pot?"}
{"text": "Rain dey make everywhere cold this evening."}
{"text": "The man dey check tyre pressure before travel."}
{"text": "No dey judge person by him cloth."}
{"text": "The girl dey help her mama sell groundnut."}
{"text": "Dog dey wag tail when e happy."}
{"text": "Na patient person dey reach destination safe."}
{"text": "The bus break down for middle of road."}
{"text": "No be all that glitter na gold."}
{"text": "The old woman dey pluck mango from tree."}
{"text": "Na small breeze dey cool body for afternoon."}
{"text": "She buy new wrapper for church thanksgiving."}
{"text": "No dey carry gossip from one place to another."}
{"text": "The man dey wash car every Saturday morning."}
{"text": "The pikin dey chase butterfly for garden."}
{"text": "Who leave tap open make water dey waste?"}
{"text": "No matter how e be, tomorrow fit better."}
{"text": "The trader dey count change under umbrella."}
{"text": "Na small lie dey turn to big problem."}
{"text": "The fisherman cast net early for morning."}
{"text": "She dey comb her hair for mirror before outing."}
{"text": "The student dey revise before exam start."}
{"text": "No use sweet mouth deceive yourself."}
{"text": "The goat dey chew paper for backyard."}
{"text": "Na peace dey make house sweet."}
{"text": "The motor dey honk near roundabout."}
{"text": "No forget to thank person wey help you."}
{"text": "The tailor sew cloth fine well well."}
{"text": "She dey sing lullaby make baby sleep."}
{"text": "No be by muscle alone man dey strong."}
{"text": "The old papa dey mend his shoe quietly."}
{"text": "Na small kindness dey open big opportunity."}
{"text": "The girl dey wash fruit before she chop am."}
{"text": "Who go help me carry this bag enter house?"}
{"text": "No dey run when you never see danger finish."}
{"text": "The farmer dey plant maize for rainy season."}
{"text": "She dey smile even when she tired."}
{"text": "No use anger spoil long friendship."}
{"text": "The boy dey whistle as he walk home."}
{"text": "Na who work hard dey enjoy better life later."}
{"text": "The driver dey check mirror before move."}
{"text": "No be every talk you must answer."}
{"text": "The market woman dey arrange pepper for tray."}
{"text": "Sun don set, make we dey go house small small."}
{"text": "The pikin dey write ABC for slate happily."}
{"text": "The woman dey tie headscarf before she go church."}
{"text": "No dey use person mistake laugh am."}
{"text": "The boy dey clean blackboard after class."}
{"text": "Na morning breeze dey sweet pass for village."}
{"text": "She dey dance for rain like small pikin."}
{"text": "The man dey fix his radio wey no dey work again."}
{"text": "No be every day goat dey escape from rope."}
{"text": "The tailor dey measure cloth with tape for table."}
{"text": "Who hide broom under bed?"}
{"text": "The old woman dey sing folk song for children."}
{"text": "Na better talk dey settle fight."}
{"text": "The boy dey play football near gutter."}
{"text": "No use person wahala take laugh yourself."}
{"text": "The woman dey grind pepper for mortar."}
{"text": "She dey wait for bus under hot sun."}
{"text": "Na who wake early dey see opportunity."}
{"text": "The farmer dey sharpen cutlass with stone."}
{"text": "No let hunger make you vex anyhow."}
{"text": "The teacher dey write new topic for board."}
{"text": "The goat dey enter kitchen go find food."}
{"text": "No be everything eye see, mouth suppose talk."}
{"text": "The boy dey mend bicycle tyre for junction."}
{"text": "Na patience dey make person grow steady."}
{"text": "The woman dey pick beans one by one."}
{"text": "No let pride block your sense."}
{"text": "The children dey play ten-ten for compound."}
{"text": "Na true friend dey stand when wahala come."}
{"text": "The driver dey shout for passenger make dem enter fast."}
{"text": "No dey borrow wetin you no fit return."}
{"text": "The man dey count his money under light."}
{"text": "Na who ask question dey sabi pass."}
{"text": "The girl dey blow balloon for birthday party."}
{"text": "No use lie build your name."}
{"text": "The tailor dey press cloth before delivery."}
{"text": "Na who carry lamp dey see road for night."}
{"text": "The boy dey sing as he dey wash plate."}
{"text": "No be every rain mean flood go come."}
{"text": "The market woman dey shout price with smile."}
{"text": "Na hope dey make person wake every morning."}
{"text": "The pikin dey follow mama go church every Sunday."}
{"text": "No forget say time no dey wait for anybody."}
{"text": "The carpenter dey nail new door frame."}
{"text": "Na small seed dey grow to big tree."}
{"text": "The man dey dry his cloth for sun."}
{"text": "No dey carry another person load for head."}
{"text": "The girl dey plait her friend hair for shade."}
{"text": "Na who sabi road fit guide traveler."}
{"text": "The goat dey drink water from basin."}
{"text": "No let jealousy spoil your mind."}
{"text": "The boy dey carry his book go school early."}
{"text": "Na laughter dey make heart light."}
{"text": "The woman dey cook jollof rice for big pot."}
{"text": "No dey use anger spoil better friendship."}
{"text": "The boy dey play draft with him papa."}
{"text": "Na morning breeze dey make sleep sweet."}
{"text": "The girl dey fetch water from well."}
{"text": "No be everything wey shine be gold."}
{"text": "The man dey wash him car for compound."}
{"text": "Na who work hard dey chop better food."}
{"text": "The pikin dey sing song for school assembly."}
{"text": "No let laziness make you miss chance."}
{"text": "The woman dey buy pepper for market stall."}
{"text": "Na small thing dey cause big wahala sometimes."}
{"text": "The farmer dey plant cassava for wet ground."}
{"text": "No forget say health na wealth."}
{"text": "The driver dey honk horn for junction."}
{"text": "Na patient person dey see result last last."}
{"text": "The tailor dey sew agbada for customer."}
{"text": "No be everything person suppose talk."}
{"text": "The girl dey read book under lamp light."}
{"text": "Na good word dey calm angry heart."}
{"text": "The man dey fix roof before rain start."}
{"text": "No let gossip turn you to enemy."}
{"text": "The pikin dey dance for children party."}
{"text": "Na small drop of rain dey cool earth."}
{"text": "The woman dey peel plantain for frying."}
{"text": "No be who laugh with you mean you well."}
{"text": "The boy dey help him papa carry firewood."}
{"text": "Na wisdom dey guide mouth from trouble."}
{"text": "The man dey check calendar for wall."}
{"text": "No forget to pray before you sleep."}
{"text": "The girl dey help her mama cook stew."}
{"text": "Na love dey make family strong."}
{"text": "The tailor dey cut cloth for small boy."}
{"text": "No use person money take joke."}
{"text": "The farmer dey harvest yam from farm."}
{"text": "Na sun dey dry cloth faster."}
{"text": "The woman dey sing praise for morning devotion."}
{"text": "No be by talk person dey show good heart."}
{"text": "The pikin dey blow whistle for playground."}
{"text": "Na who no fear work dey succeed quick."}
{"text": "The boy dey clean shoe before go school."}
{"text": "No let hunger make you do bad thing."}
{"text": "The man dey drink garri with groundnut."}
{"text": "Na respect dey open better door for person."}
{"text": "The woman dey sell akara for bus stop."}
{"text": "No be everything person see e go talk."}
{"text": "The pikin dey chase chicken for backyard."}
{"text": "Na smile dey make day bright."}
{"text": "The teacher dey write note for blackboard."}
{"text": "No use small mistake take condemn person."}
{"text": "The driver dey start engine early morning."}
{"text": "The woman dey warm soup for kitchen."}
{"text": "No dey use big grammar for market people."}
{"text": "The boy dey sweep classroom before teacher come."}
{"text": "Na morning sun dey make clothes dry quick."}
{"text": "The girl dey arrange her hair with comb."}
{"text": "No be every road lead to better place."}
{"text": "The man dey wait for bus under tree shade."}
{"text": "Na who sabi work dey make life easy."}
{"text": "The pikin dey laugh as goat dey run."}
{"text": "No forget to greet your elders for morning."}
{"text": "The tailor dey fix zip wey tear."}
{"text": "Na patient mind dey handle stubborn person."}
{"text": "The woman dey grind pepper with grinding stone."}
{"text": "No dey use lie hide truth forever."}
{"text": "The boy dey clean table after eating."}
{"text": "Na small time remain make rain start again."}
{"text": "The teacher dey mark register for class."}
{"text": "No be who tall pass dey wise pass."}
{"text": "The woman dey pack her load go village."}
{"text": "Na who respect elders dey get blessing."}
{"text": "The man dey climb ladder to fix bulb."}
{"text": "No let pride make you lose friend."}
{"text": "The girl dey sing for choir practice."}
{"text": "Na little kindness dey touch heart."}
{"text": "The pikin dey help mama wash plate."}
{"text": "No dey forget say patience be virtue."}
{"text": "The market woman dey count her profit."}
{"text": "Na good attitude dey bring opportunity."}
{"text": "The farmer dey water his crops early morning."}
{"text": "No use envy disturb your peace."}
{"text": "The boy dey draw car for his book."}
{"text": "Na laughter dey chase worry away."}
{"text": "The driver dey check engine oil before journey."}
{"text": "No let anger control your tongue."}
{"text": "The woman dey tie baby for her back."}
{"text": "Na humility dey open new chance."}
{"text": "The pikin dey recite poem for school stage."}
{"text": "No be every time person go win."}
{"text": "The girl dey pluck flower from garden."}
{"text": "Na who plan well dey enjoy peace later."}
{"text": "The man dey close shop for evening."}
{"text": "No use stubbornness spoil better opportunity."}
{"text": "The teacher dey explain maths with chalk."}
{"text": "Na peace dey make house sweet."}
{"text": "The woman dey fry fish for customers."}
{"text": "No forget say water pass garri sometimes."}
{"text": "The boy dey play with paper kite for field."}
{"text": "Na who keep secret fit hold trust."}
{"text": "The man dey polish table with rag."}
{"text": "No be every joke person suppose laugh."}
{"text": "The pikin dey eat mango under shade."}
{"text": "The woman dey set table for dinner."}
{"text": "No dey follow crowd if you no sure the road."}
{"text": "The boy dey mop floor with small bucket of water."}
{"text": "Na morning prayer dey start day well."}
{"text": "The girl dey brush her hair with mirror."}
{"text": "No be who shout pass dey get sense pass."}
{"text": "The man dey open shop early for morning."}
{"text": "Na who patient dey always get result."}
{"text": "The pikin dey help carry bag for mama."}
{"text": "No forget to close window before rain start."}
{"text": "The teacher dey correct student handwriting."}
{"text": "Na small start dey lead to big success."}
{"text": "The woman dey prepare stew with fresh tomato."}
{"text": "No dey talk anyhow for public."}
{"text": "The boy dey repair bicycle chain with hand."}
{"text": "Na good friend dey share food with joy."}
{"text": "The girl dey water flowers every morning."}
{"text": "No be everything body dey see, e fit talk."}
{"text": "The man dey read newspaper under tree."}
{"text": "Na who no give up dey reach finish line."}
{"text": "The pikin dey play ludo with him sister."}
{"text": "No forget your promise to your friend."}
{"text": "The woman dey count her tomatoes for basket."}
{"text": "Na clean environment dey keep people healthy."}
{"text": "The driver dey park bus near police checkpoint."}
{"text": "No let small anger spoil your name."}
{"text": "The girl dey tie gele for her mama wedding."}
{"text": "Na who wake early dey meet good market."}
{"text": "The boy dey read storybook for veranda."}
{"text": "No be everything wey glitter go sweet for taste."}
{"text": "The tailor dey sew school uniform for children."}
{"text": "Na love dey join family together strong."}
{"text": "The woman dey fetch water before sunrise."}
{"text": "No dey copy another person destiny."}
{"text": "The man dey sweep office floor neat."}
{"text": "Na small truth dey chase big lie away."}
{"text": "The teacher dey smile as student answer correct."}
{"text": "No forget to thank God for life."}
{"text": "The pikin dey draw house with crayon."}
{"text": "Na good behavior dey open way for person."}
{"text": "The boy dey carry tray of bread for sale."}
{"text": "No be every rain mean flood go happen."}
{"text": "The woman dey fry puff-puff for evening."}
{"text": "Na respect dey make peace easy."}
{"text": "The man dey wash his hands before chop."}
{"text": "No let greed make you fall for trap."}
{"text": "The girl dey dance cultural dance for stage."}
{"text": "Na smile dey light up dull place."}
{"text": "The teacher dey collect homework for morning."}
{"text": "No use person downfall take make joke."}
{"text": "The pikin dey sleep with teddy bear for hand."}
{"text": "E better make we plan today than to rush tomorrow."}
{"text": "Na small small bird dey build big nest."}
{"text": "If goat wan chop yam, e go wait make fire cool."}
{"text": "No be all who smile dey happy."}
{"text": "Rain don stop but road still full with water."}
{"text": "Wahala no good but sometimes e teach lesson."}
{"text": "Even tortoise dey reach where e dey go."}
{"text": "No use stone chase fly wey land for soup."}
{"text": "Sun don shine well well today."}
{"text": "Children dey play ten-ten for street."}
{"text": "Mama dey fry puff-puff for junction."}
{"text": "Papa don go farm since morning."}
{"text": "Di new teacher sabi teach English well."}
{"text": "Na who get ear go hear better advice."}
{"text": "If breeze blow, fowl yansh go open."}
{"text": "Person wey no sleep, go see night spirit."}
{"text": "Di market full today pass last week."}
{"text": "Dem dey prepare for festival next month."}
{"text": "Dog dey bark because stranger enter compound."}
{"text": "Di girl carry bucket of water for head."}
{"text": "Na patience dey make food sweet for pot."}
{"text": "River no dey forget di road go sea."}
{"text": "Monkey no fit leave tree easy."}
{"text": "Di sky clear, no sign of rain again."}
{"text": "Everybody gather for family meeting."}
{"text": "Uncle bring yam from village today."}
{"text": "If you no respect elder, you go regret am."}
{"text": "E don too late to dey shout now."}
{"text": "Time no wait for anybody at all."}
{"text": "I go try my best for this work."}
{"text": "Na who ask question go sabi road."}
{"text": "Bread and akara sweet for morning."}
{"text": "Small boy dey hawk pure water under sun."}
{"text": "Di pastor dey preach for crusade ground."}
{"text": "Motor jam for express road this evening."}
{"text": "Na who help you climb go remember you."}
{"text": "E no good make person stingy with smile."}
{"text": "Di woman dey sing hymn softly."}
{"text": "Na so life be, e no balance sometimes."}
{"text": "If pikin cry, e mama go answer am."}
{"text": "Di tailor dey measure cloth for customer."}
{"text": "E better pass to learn trade than dey idle."}
{"text": "Cow dey chew grass slowly like king."}
{"text": "Dem go soon start match for TV."}
{"text": "Na wetin man sow e go reap."}
{"text": "If rain wan fall, breeze go first show face."}
{"text": "Na every hand join make broom strong."}
{"text": "Palm wine sweet well when e fresh."}
{"text": "Di small girl dey sweep compound."}
{"text": "Dem dey build new house near school."}
{"text": "Nobody fit stop sun from shining."}
{"text": "Na better morning wey go bring good day."}
{"text": "E good make pikin sabi greet early."}
{"text": "Di old man dey tell story under tree."}
{"text": "Time go tell who get real plan."}
{"text": "Even lizard dey nod head after jump."}
{"text": "Na wetin you put for pot you go chop."}
{"text": "No dey use anger spoil your blessing."}
{"text": "Rain dey fall small small since afternoon."}
{"text": "Children dey sing for school assembly."}
{"text": "Na only hard work fit change destiny."}
{"text": "Goat no dey forget person wey beat am."}
{"text": "Di chicken don lay egg for back of house."}
{"text": "Na so dem dey do am for village too."}
{"text": "If you wan chop fruit, you go climb tree."}
{"text": "People dey gather for naming ceremony."}
{"text": "No let wahala blind your sense."}
{"text": "Na from small talk quarrel dey start."}
{"text": "Na who dey laugh last dey win game."}
{"text": "Money fit buy bed but e no fit buy sleep."}
{"text": "If road no straight, driver go slow."}
{"text": "E better make man honest than rich thief."}
{"text": "Bird wey no sabi sing go fly away."}
{"text": "Dem dey dig well for backyard now."}
{"text": "Sun dey hot pass fire today."}
{"text": "If you plant maize, no expect yam harvest."}
{"text": "E no get how long, one day go clear."}
{"text": "Small pikin don learn how to ride bicycle."}
{"text": "No be every laugh mean say person happy."}
{"text": "Na only God know tomorrow matter."}
{"text": "Di hunter dey clean gun for bush path."}
{"text": "Na so life dey teach man sense."}
{"text": "Fish no dey survive outside water."}
{"text": "Di woman dey peel yam for dinner."}
{"text": "Man wey waka fast go reach before others."}
{"text": "If fire no dey, smoke no go show."}
{"text": "Na palm oil make soup red."}
{"text": "No use sweet mouth deceive yourself."}
{"text": "Time don reach to face real work."}
{"text": "Di baby dey sleep for him mama lap."}
{"text": "Na who sabi road fit show another."}
{"text": "If your hand clean, you no go fear shake."}
{"text": "Dem dey play football for field now."}
{"text": "Na morning breeze sweet pass for village."}
{"text": "E go good make we rest small today."}
{"text": "No forget your umbrella, sky dey dark."}
{"text": "People dey dance for street carnival."}
{"text": "Na so dem take begin market business."}
{"text": "If your heart clean, peace go dey your mind."}
{"text": "E no get how far, make we just start small."}
{"text": "Di moon don full for sky tonight."}
{"text": "Person wey no work no go chop."}
{"text": "If pikin stubborn, cane dey wait am."}
{"text": "Na small drops of water dey form ocean."}
{"text": "Man wey patient go chop ripe fruit."}
{"text": "E good make person dey talk truth always."}
{"text": "No let jealousy enter your heart."}
{"text": "Di teacher dey mark exam paper now."}
{"text": "E go rain soon, make you carry cloth enter."}
{"text": "Na who dey farm go get food for house."}
{"text": "Even poor man fit laugh from heart."}
{"text": "E don reach time to cook dinner."}
{"text": "Children dey watch cartoon for parlor."}
{"text": "Na breeze from sea dey cool town today."}
{"text": "Di new baby fine like morning flower."}
{"text": "No use anger spoil friendship."}
{"text": "Na good mind make life sweet."}
{"text": "If you fall, stand up again."}
{"text": "Di driver blow horn near junction."}
{"text": "E no easy but God go help person."}
{"text": "Rain wash everywhere clean for morning."}
{"text": "Di pastor pray for family blessing."}
{"text": "If goat no get rope, e go waka anywhere."}
{"text": "Person wey no sabi go learn one day."}
{"text": "Di young boy dey sell banana for roadside."}
{"text": "Na small talk dey cause big fight sometimes."}
{"text": "E good make we dey thankful every day."}
{"text": "Sun dey shine bright like gold."}
{"text": "Na woman hand dey make house neat."}
{"text": "If fish swim well, e no go enter trap."}
{"text": "Di old man dey sleep for chair outside."}
{"text": "Na who get sense go stay out of trouble."}
{"text": "E good to share food with hungry person."}
{"text": "Na God hand dey protect traveller for road."}
{"text": "Market woman dey shout price for yam."}
{"text": "If rain start, people go run enter shade."}
{"text": "No dey envy your friend progress."}
{"text": "E better late than never at all."}
{"text": "Na small tin dey spoil better name."}
{"text": "If pot too full, soup go pour for fire."}
{"text": "E sweet to rest after long day work."}
{"text": "Person wey chop salt go sabi taste."}
{"text": "If you see snake, no go play with am."}
{"text": "Di cat dey hide under chair quietly."}
{"text": "Na so we take begin our hustle."}
{"text": "E go make sense if we plan together."}
{"text": "No let small success make you proud."}
{"text": "E dey better to forgive than fight."}
{"text": "Na morning dew make grass fresh."}
{"text": "If dog bark too much, e dey warn."}
{"text": "Di woman carry baby for back dey pound yam."}
{"text": "E better to talk less and think more."}
{"text": "Na new day, new chance for blessing."}
{"text": "If you respect people, dem go respect you."}
{"text": "Di boy dey play drum for church today."}
{"text": "No be every friend go happy for your win."}
{"text": "E no good make pikin lie to mama."}
{"text": "Na patience dey open every closed door."}
{"text": "Sun dey go down for west side now."}
{"text": "Di man buy plantain for evening chop."}
{"text": "If tree tall, wind go still shake am."}
{"text": "Na who no fear wahala dey talk anyhow."}
{"text": "E better make we rest small for shade."}
{"text": "Di small girl dey dance to radio song."}
{"text": "If person too proud, fall dey near."}
{"text": "No dey forget people wey help you."}
{"text": "E sweet when family gather for meal."}
{"text": "Na today market full pass this week."}
{"text": "If your shoe tight, you no go fit waka far."}
{"text": "Di tailor promise say cloth go ready soon."}
{"text": "E no get secret wey time no go show."}
{"text": "Na rain water dem dey use wash car."}
{"text": "Di young man dey play football with friends."}
{"text": "If hunger catch you, even garri go sweet."}
{"text": "Na who hear word go stay safe."}
{"text": "Di chief dey greet everybody for meeting."}
{"text": "No dey forget where you start from."}
{"text": "E good to pray before you sleep."}
{"text": "Na so story take begin that year."}
{"text": "If pepper too much, soup go hot well."}
{"text": "Di man dey repair radio for shop."}
{"text": "E better use sense than shout for nothing."}
{"text": "No let laziness spoil your future."}
{"text": "Di mother dey bath baby for basin."}
{"text": "Na morning bell wake students up."}
{"text": "If wind strong, tree go bend small."}
{"text": "Na every new moon get own story."}
{"text": "E go rain again this evening sure."}
{"text": "Di hunter return from bush with antelope."}
{"text": "No let anger spoil better day."}
{"text": "Na who hold torch dey see road for night."}
{"text": "E good to mind your own business."}
{"text": "Di teacher call names for attendance."}
{"text": "Na true talk dey heal heart faster."}
{"text": "If person wise, e go avoid trouble."}
{"text": "E sweet to see friends after long time."}
{"text": "Na morning breeze dey fresh pass anything."}
{"text": "If you no try, you no go know result."}
{"text": "Di village square full with dancers tonight."}
{"text": "Na hard work dey bring respect for man."}
{"text": "E no get how far, make we start small small."}
{"text": "The mama dey wash cloth for backyard."}
{"text": "Na God hand dey protect person for road."}
{"text": "If you no hear word, wahala go teach you."}
{"text": "Di pikin dey cry because e hungry."}
{"text": "E better make we plan before we act."}
{"text": "The woman dey plait hair for corner."}
{"text": "Na who sabi cook go get visitor everyday."}
{"text": "E good to rest after long day hustle."}
{"text": "The teacher dey correct student handwriting."}
{"text": "Na breeze from window dey cool room."}
{"text": "E better to forgive than carry grudge."}
{"text": "The small boy dey help mama sweep house."}
{"text": "If rain too much, flood go enter road."}
{"text": "Na who no plan well go regret later."}
{"text": "The driver park bus for junction dey rest."}
{"text": "E good make you thank person wey help you."}
{"text": "The goat don jump fence go neighbor farm again."}
{"text": "Na patience dey make palm oil sweet."}
{"text": "If you no try, success go pass you by."}
{"text": "The cat dey chase small rat for kitchen."}
{"text": "Na small word fit break big friendship."}
{"text": "The boy dey play ball for front of house."}
{"text": "E good make man dey humble always."}
{"text": "The woman buy pepper and onion for market."}
{"text": "Na small talk dey cause big wahala sometimes."}
{"text": "The small girl dey help mama sell pure water."}
{"text": "E no get how you hide, truth go show one day."}
{"text": "Na evening breeze dey sweet pass for village."}
{"text": "The man dey sharpen knife for kitchen."}
{"text": "No dey use person condition mock am."}
{"text": "E better make you talk true than lie."}
{"text": "The boy dey carry load go bus stop."}
{"text": "Na hard work dey bring good name."}
{"text": "E good make person dey pray before sleep."}
{"text": "The tailor don measure my cloth finish."}
{"text": "No be every laugh mean say person happy."}
{"text": "Na rain wash all the dirt from road today."}
{"text": "The baby dey smile when mama sing lullaby."}
{"text": "E good make pikin respect elders."}
{"text": "The man dey fix chair wey break yesterday."}
{"text": "No dey forget the people wey help you climb."}
{"text": "Na morning food dey give strength for work."}
{"text": "The girl dey dance for school competition."}
{"text": "E no good to shout for small mistake."}
{"text": "The farmer dey plant corn for dry season."}
{"text": "Na who sabi wait go see better day."}
{"text": "The market full well today with people."}
{"text": "E better make you listen before you talk."}
{"text": "The student dey read for exam inside library."}
{"text": "Na prayer dey protect man from evil."}
{"text": "The mother dey cook egusi soup for lunch."}
{"text": "E no matter where you from, respect still count."}
{"text": "The small boy dey fetch water for compound."}
{"text": "No let pride make you lose friendship."}
{"text": "E better learn now than regret tomorrow."}
{"text": "The woman dey sew cloth with her old machine."}
{"text": "Na who no fear failure fit win big."}
{"text": "E good make we dey help each other for street."}
{"text": "The rain don stop, but ground still wet."}
{"text": "No use anger talk wetin you go regret."}
{"text": "The pikin dey learn how to write ABC."}
{"text": "E better to save small money every day."}
{"text": "Na who respect elders dey get blessing."}
{"text": "The man dey fix light for compound."}
{"text": "E good make pikin go school early."}
{"text": "The tailor dey work for customer night and day."}
{"text": "No be every road wey fine dey safe."}
{"text": "E better to keep quiet than talk nonsense."}
{"text": "The woman dey wash plate for backyard."}
{"text": "Na morning breeze dey sweet pass anything."}
{"text": "E no matter how night dark, morning go come."}
{"text": "The man dey clean car before work."}
{"text": "Na prayer dey open closed door."}
{"text": "E better make we rest small after food."}
{"text": "The boy dey count change after buy sweet."}
{"text": "Na good name better pass silver and gold."}
{"text": "E no easy but hard work go pay."}
{"text": "The mama dey call pikin come chop."}
{"text": "No let envy spoil your peace of mind."}
{"text": "E better to talk small and wise."}
{"text": "The small boy dey run go buy bread."}
{"text": "Na evening time dem dey gist for compound."}
{"text": "E no matter how slow, just keep moving."}
{"text": "The teacher dey write note for blackboard."}
{"text": "Na smile dey make person fine pass."}
{"text": "E better to learn from mistake."}
{"text": "The pikin dey dance when music play."}
{"text": "Na old man wisdom dey help young people."}
{"text": "E no get how e be, life still sweet small."}
{"text": "The driver dey wait for passengers for park."}
{"text": "Na truth dey give peace for heart."}
{"text": "E better make you thank God for today."}
{"text": "The woman dey fry akara early morning."}
{"text": "Na small word fit cause big fight."}
{"text": "E no easy to work under hot sun."}
{"text": "The baby dey sleep for mama lap."}
{"text": "Na friendship dey make life sweet."}
{"text": "E good to share food with neighbor."}
{"text": "The hunter don return from bush with meat."}
{"text": "Na who patient go reap good thing."}
{"text": "E no matter how far, one day you go reach."}
{"text": "The woman dey pound yam for kitchen this evening."}
{"text": "E no get how life hard, hope still dey."}
{"text": "Na who get sense dey avoid wahala."}
{"text": "The boy dey fetch water from borehole."}
{"text": "E better make you rest before night reach."}
{"text": "The tailor dey cut new cloth pattern."}
{"text": "Na who sabi wait go chop ripe mango."}
{"text": "E good make person dey honest always."}
{"text": "The man dey repair his radio for workshop."}
{"text": "Na who early wake up dey see morning dew."}
{"text": "E better to give than to collect every time."}
{"text": "The mama dey help her neighbor wash baby cloth."}
{"text": "Na small fire dey cook big pot of soup."}
{"text": "E no matter how rain fall, sun go shine later."}
{"text": "The teacher dey correct spelling for blackboard."}
{"text": "Na who no fear work go enjoy later."}
{"text": "E better make you smile than frown always."}
{"text": "The woman dey sweep veranda with broom."}
{"text": "Na respect dey make person get peace for house."}
{"text": "E no good to talk bad about people."}
{"text": "The farmer dey carry cassava go market."}
{"text": "Na small step dey lead to long journey."}
{"text": "E better to plan today than rush tomorrow."}
{"text": "The small boy dey learn how to ride bicycle."}
{"text": "Na who no respect time go lose chance."}
{"text": "E good to forgive fast and move on."}
{"text": "The mama dey cook soup with firewood."}
{"text": "Na prayer dey protect traveler for road."}
{"text": "E better make person patient when waiting."}
{"text": "The pikin dey smile when papa come back."}
{"text": "Na who sabi listen go learn new thing."}
{"text": "E no matter how far, God eye still dey watch."}
{"text": "The woman dey pack tomato for basket."}
{"text": "Na better behavior dey open door for favor."}
{"text": "E good to wash hand before you chop food."}
{"text": "The man dey wait for his turn to talk."}
{"text": "Na who plant today go harvest tomorrow."}
{"text": "E better to work small small than rush."}
{"text": "The teacher dey tell students moral story."}
{"text": "Na peace dey make house sweet to stay."}
{"text": "E no good make person lazy for work."}
{"text": "The driver don fix tire wey burst for road."}
{"text": "Na who no plan dey complain later."}
{"text": "E better make we share idea together."}
{"text": "The girl dey write letter to her friend."}
{"text": "Na good talk dey calm angry person."}
{"text": "E better make you prepare early for exam."}
{"text": "The woman dey roast corn under tree."}
{"text": "Na who fear God go live long."}
{"text": "E no good to gossip about your neighbor."}
{"text": "The mama dey sing lullaby for her baby."}
{"text": "Na who work hard go see reward soon."}
{"text": "E better to learn from mistake than hide am."}
{"text": "The small girl dey plait doll hair."}
{"text": "Na clean environment dey prevent sickness."}
{"text": "E good make person dey truthful for business."}
{"text": "The man dey build new fence for compound."}
{"text": "Na who patient go collect full blessing."}
{"text": "E no matter how night dark, joy go come."}
{"text": "The boy dey fry egg for breakfast."}
{"text": "Na respect make two friends still dey talk."}
{"text": "E better make we calm before we react."}
{"text": "The student dey write test for classroom."}
{"text": "Na who keep quiet dey avoid trouble."}
{"text": "E good to share knowledge with others."}
{"text": "The woman dey carry firewood go house."}
{"text": "Na small kindness fit change person life."}
{"text": "E better to learn slowly than no learn at all."}
{"text": "The farmer dey clear bush for new farm."}
{"text": "Na patience dey make fruit ripe."}
{"text": "E no matter how sun hot, rain go still fall."}
{"text": "The mama dey mend her pikin cloth."}
{"text": "Na who listen to advice dey progress."}
{"text": "E better to talk small than argue plenty."}
{"text": "The pikin dey arrange toys for floor."}
{"text": "Na morning prayer dey start good day."}
{"text": "E good make person dey thankful for small wins."}
{"text": "The man dey buy bread and milk for breakfast."}
{"text": "Na who strong heart dey win battle."}
{"text": "E better make we settle matter peacefully."}
{"text": "The old woman dey sit for veranda dey rest."}
{"text": "Na time dey show who true friend be."}
{"text": "E good to greet elders for morning."}
{"text": "The teacher dey share note for students."}
{"text": "Na clean hand dey make food sweet to eat."}
{"text": "E better make we no give up for dream."}
{"text": "The woman dey arrange market goods for sale."}
{"text": "Na who humble dey get plenty friend."}
{"text": "E good make you mind your own business."}
{"text": "The pikin dey draw picture for wall with chalk."}
{"text": "Na morning sun dey give life to plants."}
{"text": "E no matter the weather, people go survive."}
{"text": "The small girl dey sing song for school."}
{"text": "Na love dey hold family together."}
{"text": "E better make person talk true always."}
{"text": "The farmer dey carry yam enter truck."}
{"text": "Na prayer dey open closed doors."}
{"text": "E no good make pikin talk lie to mama."}
{"text": "The man dey arrange chairs for meeting."}
{"text": "Na peace of mind pass money sometimes."}
{"text": "E good to say thank you even for small thing."}
{"text": "Di tailor dey sew new cloth for customer."}
{"text": "E no easy to wake early every morning."}
{"text": "Children dey dance for church compound."}
{"text": "Monkey no fit forget how to climb tree."}
{"text": "Di goat don escape from di pen again."}
{"text": "Mama dey fry akara for junction every Saturday."}
{"text": "Breeze just blow my paper commot for table."}
{"text": "Driver dey shout make people enter bus quick."}
{"text": "My brother don get new job for Lagos."}
{"text": "No be every friend wey smile dey mean well."}
{"text": "Di house dey close to big mango tree."}
{"text": "Rain don scatter all di clothes for line."}
{"text": "Na small small we go reach our destination."}
{"text": "Di man dey repair radio for corner shop."}
{"text": "Cat don chop fish wey we keep for kitchen."}
{"text": "If you no chop, how you go get strength?"}
{"text": "Di pikin dey learn how to read book."}
{"text": "My shoe don tear for road again."}
{"text": "Dem dey play football match for school field."}
{"text": "Market woman dey shout make people buy tomato."}
{"text": "No be every time person go dey talk."}
{"text": "Di teacher dey explain new topic for class."}
{"text": "Sun too hot today, even stone dey sweat."}
{"text": "Papa say make we no go outside till night."}
{"text": "Dog dey bark since morning for compound."}
{"text": "My friend don travel go Port Harcourt."}
{"text": "Na only patient person fit win this race."}
{"text": "Light don come back after two days of darkness."}
{"text": "Di chair don break because e old."}
{"text": "Nobody fit carry water for basket."}
{"text": "Di woman dey tie headscarf with bright color."}
{"text": "We go reach there before rain start."}
{"text": "Time don reach to pack all di yam inside."}
{"text": "My uncle dey sell phone for computer village."}
{"text": "Dem don paint di house yellow again."}
{"text": "No matter how e be, we go survive."}
{"text": "Di palm wine sweet pass any drink for here."}
{"text": "Every Sunday, di choir dey practice new song."}
{"text": "E better make you late than make you no show."}
{"text": "Di hunter catch big antelope for bush."}
{"text": "My auntie dey bake bread for living."}
{"text": "Nobody wan do di dirty work for now."}
{"text": "Di sky don dark small small, rain go soon fall."}
{"text": "Dem dey repair di bridge wey break last week."}
{"text": "E no good make you dey gossip your friend."}
{"text": "Di coconut water dey cool for throat."}
{"text": "Na small mosquito dey spoil big sleep."}
{"text": "My mama always tell me make I pray before sleep."}
{"text": "Di landlord say rent go increase next month."}
{"text": "No rush life, everything get time."}
{"text": "Goat dey chop yam for backyard like say e pay rent."}
{"text": "Di mechanic still dey fix motor since morning."}
{"text": "No forget say patience dey bring better result."}
{"text": "Di soup don too pepper, e go make person cry."}
{"text": "Children dey chase butterfly for field."}
{"text": "Na today be di wedding wey everybody don wait for."}
{"text": "My phone battery don die, I no fit call again."}
{"text": "E dey hard to find good friend for this life."}
{"text": "Di baby dey laugh anytime she see her papa face."}
{"text": "No use anger spoil your day."}
{"text": "I dey wait bus for junction since afternoon."}
{"text": "Di plantain wey we fry yesterday still remain small."}
{"text": "Di goat no wan enter house again."}
{"text": "E better make person tell truth from beginning."}
{"text": "Rain dey fall small small like blessing."}
{"text": "My mama don buy rice for Sunday food."}
{"text": "Di pastor talk say make we love one another."}
{"text": "Na who wake early go catch better thing."}
{"text": "E no go good if you forget where you come from."}
{"text": "Di tailor dey iron cloth for customer."}
{"text": "Monkey dey climb coconut tree for back of house."}
{"text": "E don too late to go market now."}
{"text": "Di teacher don mark all di exam papers."}
{"text": "Na my papa build dis fence with him hand."}
{"text": "Water dey boil for pot since morning."}
{"text": "No use person downfall take laugh."}
{"text": "Di old man dey tell story for fire side."}
{"text": "My pikin dey fear darkness for night."}
{"text": "E get some truth wey dey hard to talk."}
{"text": "Di radio don spoil again, e no dey play sound."}
{"text": "We don cook plenty rice for di party."}
{"text": "Di mango tree full with ripe fruit."}
{"text": "No forget say tomorrow still dey."}
{"text": "Di bus driver dey collect change slow slow."}
{"text": "My sister dey learn hairdressing for shop."}
{"text": "Na only God know tomorrow."}
{"text": "Di market full with people wey dey shout price."}
{"text": "We go reach home before sun go set."}
{"text": "No allow fear stop your dream."}
{"text": "E don tay wey I see you last."}
{"text": "Di carpenter dey fix table wey break."}
{"text": "Di woman dey sweep compound early morning."}
{"text": "No use lie cover another lie."}
{"text": "My uncle don return from abroad yesterday."}
{"text": "Di goat dey hide under table when e rain."}
{"text": "Na di small thing dey cause big wahala."}
{"text": "We dey plan to travel next weekend."}
{"text": "Di boy dey wash motor for street."}
{"text": "E no get who no dey make mistake."}
{"text": "Di dog dey wag tail anytime e happy."}
{"text": "No forget your umbrella, rain go soon start."}
{"text": "My neighbor dey play loud music for night."}
{"text": "Na hustle go make man chop better food."}
{"text": "Di church bell dey ring every Sunday morning."}
{"text": "E no good make person dey proud."}
{"text": "Di yam wey we plant don begin grow."}
{"text": "No worry, time go tell everything."}
{"text": "Di woman dey cook stew with fresh tomato."}
{"text": "My brother dey fix light bulb for ceiling."}
{"text": "E no easy to dey patient for hard time."}
{"text": "Di old man dey sit near window dey watch road."}
{"text": "If you no study, exam go hard you."}
{"text": "Na who get ear suppose listen well."}
{"text": "Di pikin dey play ball for backyard."}
{"text": "Rain don make everywhere cold this night."}
{"text": "No carry person matter put for your head."}
{"text": "Di tailor don sew my cloth finish."}
{"text": "Di fisherman catch big tilapia today."}
{"text": "My mama dey sing while she dey cook."}
{"text": "No forget say tomorrow get im own wahala."}
{"text": "Di driver dey hon horn too much."}
{"text": "My auntie dey teach small children for school."}
{"text": "Na small lie dey cause big problem."}
{"text": "Di soup don sweet after we add crayfish."}
{"text": "E get time wey everything go balance."}
{"text": "Di man dey paint di house white."}
{"text": "No let anybody use you play."}
{"text": "My shoe don old, I need new one."}
{"text": "Di road to village no smooth again."}
{"text": "Na prayer go change person story."}
{"text": "Di woman dey wash plate for back of house."}
{"text": "E better make you learn work than beg."}
{"text": "My papa dey rest after long day for farm."}
{"text": "No use eye look down on poor person."}
{"text": "Di trader dey count money for table."}
{"text": "E no good to borrow too much."}
{"text": "Di plant dey grow well because of rain."}
{"text": "We don reach bus stop finally."}
{"text": "My brother dey learn mechanic work."}
{"text": "Na today be di final exam for class."}
{"text": "E no make sense to fight over small thing."}
{"text": "Di police dey control traffic for junction."}
{"text": "Di goat dey chew paper for veranda."}
{"text": "We go rest small before we continue journey."}
{"text": "No let money control your mind."}
{"text": "Di baby don sleep for mama lap."}
{"text": "E get some secret wey better make we no talk."}
{"text": "My friend dey fix his bicycle chain."}
{"text": "Na di same sun dey shine for everybody."}
{"text": "E go better for who dey work hard."}
{"text": "Di woman dey dry pepper under sun."}
{"text": "No forget say life na small small."}
{"text": "My papa dey read newspaper every evening."}
{"text": "E good make person dey thankful for small thing."}
{"text": "My sister dey sell fruit for junction."}
{"text": "No run before you learn how to walk."}
{"text": "Di pastor dey preach about peace today."}
{"text": "My friend buy new slippers for market."}
{"text": "E no easy to wake early when cold dey."}
{"text": "Di pikin dey cry because e hungry."}
{"text": "Na so life be, sometimes sweet, sometimes bitter."}
{"text": "My uncle dey repair radio for shop."}
{"text": "Di teacher dey mark book under tree."}
{"text": "No worry, tomorrow go better pass today."}
{"text": "My brother dey play guitar for church."}
{"text": "E get one cat wey like to climb tree."}
{"text": "Di woman dey grind pepper with stone."}
{"text": "Na love go make family stay together."}
{"text": "E no make sense to dey shout for no reason."}
{"text": "Di rain don spoil road to market."}
{"text": "My papa dey clean him motorcycle every morning."}
{"text": "No forget where you come from."}
{"text": "Di children dey run for compound."}
{"text": "E better make person dey honest always."}
{"text": "My mama dey fry puff-puff for corner."}
{"text": "Na early bird dey catch worm."}
{"text": "Di old woman dey tell story for children."}
{"text": "No judge person by him cloth."}
{"text": "My friend dey fix fence wey fall."}
{"text": "E no good to gossip about people."}
{"text": "Di man dey sweep floor with broom."}
{"text": "Na patient person dey enjoy result later."}
{"text": "My sister dey draw picture of flower."}
{"text": "E good make person dey share with others."}
{"text": "Di carpenter dey build chair for customer."}
{"text": "No forget to lock door before you sleep."}
{"text": "My papa don travel go town."}
{"text": "E get one song wey dey remind me of home."}
{"text": "Di sky dey clear after big rain."}
{"text": "No put blame for another person head."}
{"text": "My neighbor dey wash clothes for tap."}
{"text": "E better make person learn from mistake."}
{"text": "Di chicken dey find food for backyard."}
{"text": "My friend dey write letter to him mama."}
{"text": "E no good to waste food."}
{"text": "Di tailor dey measure cloth for new customer."}
{"text": "Na small small we dey learn big thing."}
{"text": "My brother dey play ball for street."}
{"text": "E no easy to find work these days."}
{"text": "Di woman dey carry firewood for head."}
{"text": "No forget to greet elder when you see am."}
{"text": "My papa dey repair roof before rain start."}
{"text": "E good make person dey pray every day."}
{"text": "Di goat don enter my farm again."}
"""

In [4]:
# The data to be written to the file.
# PASTE YOUR 1.5K LINES OF JSONL TEXT INSIDE THE TRIPLE QUOTES BELOW.
# I've included your sample data as a placeholder.

# The name of the file you want to create.
file_path = "pidgin_cpt.jsonl"

# Write the data to the file.
# The 'w' mode means "write", which will create the file or overwrite it if it already exists.
try:
    with open(file_path, "w") as f:
        # The strip() removes any leading/trailing whitespace from your pasted block.
        f.write(jsonl_data.strip())
    print(f"✅ Successfully created the file '{file_path}'")
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ Successfully created the file 'pidgin_cpt.jsonl'


In [8]:
# ==============================================================================
# CELL 4 (Corrected): Create the 80/20 Pidgin-English Mix using allenai/c4
# ==============================================================================
import json
import random
import os
from datasets import load_dataset

# --- Configuration ---
pidgin_file_path = "pidgin_cpt.jsonl"
combined_file_path = "pidgin_english_cpt_80_20.jsonl"
# BEST PRACTICE: Update the cache file name to match the new data source
english_sample_path = "c4_en_sample_for_pidgin.jsonl"

# --- 1. Load your local Pidgin data ---
print(f"Loading local Pidgin data from '{pidgin_file_path}'...")
with open(pidgin_file_path, "r") as f:
    pidgin_data = [json.loads(line) for line in f]
num_pidgin_samples = len(pidgin_data)
print(f"✅ Found {num_pidgin_samples} Pidgin samples.")

# --- 2. Calculate and fetch the English data ---
num_english_samples = num_pidgin_samples // 4
print(f"For an 80/20 mix, we need {num_english_samples} English samples.")

english_data = []

# --- Check for a cached sample ---
if os.path.exists(english_sample_path):
    print(f"Found cached English sample at '{english_sample_path}'. Loading from file.")
    with open(english_sample_path, "r") as f:
        english_data = [json.loads(line) for line in f]
else:
    print("No cached sample found. Downloading a new sample from 'allenai/c4'...")
    try:
        # Load the C4 dataset stream
        c4_stream = load_dataset(
           "allenai/c4",
           "en",  # Specify the English configuration
           split="train",
           streaming=True
        )

        # FIX: Iterate over the correct variable name 'c4_stream'
        for example in c4_stream.take(num_english_samples):
            english_data.append({"text": example["text"]})

        # Save the sample to a local file for future runs
        with open(english_sample_path, "w") as f:
            for item in english_data:
                f.write(json.dumps(item) + "\n")
        print(f"✅ Downloaded and cached {len(english_data)} English samples to '{english_sample_path}'.")

    except Exception as e:
        print(f"❌ An error occurred while downloading the dataset: {e}")
        print("This might be a temporary issue with the Hugging Face Hub. Please try again later.")

# --- 3. Combine and Shuffle if English data was successfully loaded ---
if english_data:
    print("\nCombining and shuffling Pidgin and English data...")
    combined_data = pidgin_data + english_data
    random.shuffle(combined_data)
    print(f"✅ Created a combined dataset with {len(combined_data)} total samples.")

    # --- 4. Save the new combined dataset ---
    print(f"Saving the final mixed dataset to '{combined_file_path}'...")
    with open(combined_file_path, "w") as f:
        for item in combined_data:
            f.write(json.dumps(item) + "\n")

    print("🎉 All done! Your dataset is now 'well-equipped' according to the paper's findings.")
else:
    print("\nSkipping dataset combination due to download failure.")

Loading local Pidgin data from 'pidgin_cpt.jsonl'...
✅ Found 1540 Pidgin samples.
For an 80/20 mix, we need 385 English samples.
No cached sample found. Downloading a new sample from 'allenai/c4'...


Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

✅ Downloaded and cached 385 English samples to 'c4_en_sample_for_pidgin.jsonl'.

Combining and shuffling Pidgin and English data...
✅ Created a combined dataset with 1925 total samples.
Saving the final mixed dataset to 'pidgin_english_cpt_80_20.jsonl'...
🎉 All done! Your dataset is now 'well-equipped' according to the paper's findings.


In [9]:
# ==============================================================================
# CELL 5: Upload the Final Combined Dataset to Hugging Face Hub
# ==============================================================================
from huggingface_hub import create_repo, upload_file

# --- Configuration ---
repo_id = "surfiniaburger/orpheus-pidgin-cpt"
new_local_file = "pidgin_english_cpt_80_20.jsonl"
path_in_repo = f"data/{new_local_file}" # A good practice to keep data in a 'data' folder

# --- 1. Ensure the dataset repository exists ---
print(f"Ensuring repository '{repo_id}' exists...")
create_repo(
    repo_id=repo_id,
    repo_type="dataset",
    exist_ok=True  # This is safe to run multiple times
)
print(f"✅ Repository is ready.")

# --- 2. Upload the new, combined file ---
print(f"Uploading '{new_local_file}' to '{repo_id}'...")
try:
    upload_file(
        path_or_fileobj=new_local_file,
        path_in_repo=path_in_repo,
        repo_id=repo_id,
        repo_type="dataset",
    )
    print(f"✅ File uploaded successfully! Find it at: https://huggingface.co/datasets/{repo_id}/blob/main/{path_in_repo}")
except Exception as e:
    print(f"❌ An error occurred during upload: {e}")

Ensuring repository 'surfiniaburger/orpheus-pidgin-cpt' exists...
✅ Repository is ready.
Uploading 'pidgin_english_cpt_80_20.jsonl' to 'surfiniaburger/orpheus-pidgin-cpt'...
✅ File uploaded successfully! Find it at: https://huggingface.co/datasets/surfiniaburger/orpheus-pidgin-cpt/blob/main/data/pidgin_english_cpt_80_20.jsonl
